In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# Origens (Silver)
df_chamado = spark.read.table("v_credit.silver.tb_chamado")
df_pesquisa = spark.read.table("v_credit.silver.tb_pesquisa")

# Destino (Gold)
TABELA_DESTINO = "v_credit.gold.ft_performance_agente"

In [0]:
# Lógica Granular (Sem GroupBy)
df_gold = (
    df_chamado.alias("c")
    .join(df_pesquisa.alias("p"), on="cd_chamado", how="left")
    # Filtra apenas humanos (cd_atendente preenchido)
    .filter(F.col("c.cd_atendente").isNotNull())
    .select(
        # PK Hash (Neste caso, pode ser o próprio cd_chamado ou um hash dele, pois o grão é chamado)
        F.md5(F.concat(F.col("c.cd_chamado").cast("string"), F.lit("PERF"))).alias("pk_performance"),
        
        F.col("c.cd_atendente"),
        F.to_date("c.dt_ingestion").alias("dt_referencia"),
        
        # Métricas Individuais (Power BI vai somar/médiar depois)
        F.lit(1).cast("bigint").alias("qtd_chamados_atendidos"), # Cada linha vale 1
        F.col("c.tm_duracao").cast("bigint").alias("val_tempo_logado_seg"),
        F.col("p.nu_nota").alias("val_nota_media"), # Nota do chamado individual
        
        # Linhagem (Direto da Silver, sem Max/Min pois não tem agregação)
        F.col("c.dt_ingestion"),
        F.col("c.dc_origem")
    )
)

# Merge (Upsert)
DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.pk_performance = s.pk_performance"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Fato Performance Agente carregada (Granular)!")